In [1]:
import sys
!{sys.executable} -m pip install praw

  Using cached https://files.pythonhosted.org/packages/a4/d7/0c7fb864ab9469f2964b2787910c78ade6d98c2d1710df1deab61beb99a1/praw-6.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cd/6d/4bf4d65d4771b15c70284c0626d24f230a5de6edac75c446fd22008ddbf6/prawcore-1.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/17/c9/ab11855af164d03be0ff4fddd4c46a5bd44799a9ecc1770e01a669c21168/update_checker-0.16-py2.py3-none-any.whl
    100% |████████████████████████████████| 92kB 1.9MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
    100% |████████████████████████████████| 143kB 3.9MB/s ta 0:00:01


In [66]:
import sys
!{sys.executable} -m pip install urllib

  Could not find a version that satisfies the requirement urllib (from versions: )
No matching distribution found for urllib


In [1]:
import praw
import pandas as pd
import datetime as dt
import tensorflow as tf

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
reddit = praw.Reddit(client_id='zL5PqCkuHioUcg', \
                     client_secret='QxbjMKys9v77wBfqj8Zbitb-CnM', \
                     user_agent='Headliner', \
                     username='millireturns', \
                     password='laser2')

In [186]:
subreddit = reddit.subreddit('politics')



In [348]:
new_subreddit = subreddit.hot(limit=100000)
#rerun with every option to build corpus

In [349]:
#headlinelist=[]


In [350]:
for submission in new_subreddit:
    headlinelist.append(submission.title)
    


In [351]:
headlinelist[2]
len(headlinelist)

6315

In [352]:
print(len(set(headlinelist)))
headlineset=set(headlinelist)
headlinestring=str(headlineset)
print(headlinestring)

1980
{'DNC votes to limit influence of superdelegates in presidential nominating process . ', 'Donald Trump Jr. calls a reporter who helped uncover Watergate a "leftist hack" for his Trump Tower meeting story', 'Parkland Students Call Out Florida Politicians, NRA After Jacksonville Shooting: ‘Enough of Your BS’', 'Trump’s tenure in the White House is a disappointment and a disgrace . ', 'Democrats dig for interpreter notes, policy directives on Trump-Putin summit . ', 'Illinois Sen. Tammy Duckworth On Her First Meeting With Sen. John McCain', 'Sessions: ‘Courage, determination and relentless drive’ made McCain a ‘great leader’ . ', "McCain's last message read aloud as Washington returns to work without him", 'Sessions: ‘Courage, determination and relentless drive’ made McCain a ‘great leader’', 'McCain remembered for bipartisanship, decisive Obamacare vote . ', "A warrior in every sense of the word': Politicians react to news McCain discontinuing treatment . ", 'Character still counts:

In [353]:
#topicstr=""

#for submission in subreddit.hot(limit=10000):
    #topics_dict["title"].append(submission.title)
    #topics_dict["score"].append(submission.score)
    #topicstr+=submission.title+"     "

In [354]:
#topics_data = pd.DataFrame(topics_dict)
#topics_data.title[5]
#topicstr=topicstr.rstrip()
#topicstr

In [89]:
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)

TensorFlow version: 1.4.1


In [90]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [91]:
#text=(topics_data.title.to_string(header=False, index=False))

In [355]:

text=headlinestring.lower()
print(headlinestring[1115:1200])

w enforcement by condemning 'flipping' of criminals like Cohen", 'TPUSA’s University 


In [356]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}


In [357]:
X = []
Y = []
length = len(text)
seq_length = 100
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [358]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [359]:
model = Sequential()
model.add(LSTM(700, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [360]:

model.fit(X_modified, Y_modified, epochs=7, batch_size=50)

model.save_weights('text_generator_gigantic.h5')

Epoch 1/7
151008/151008 [==============================] - 41150s 273ms/step - loss: 2.8653
Epoch 2/7
151008/151008 [==============================] - 41178s 273ms/step - loss: 2.5748
Epoch 3/7
151008/151008 [==============================] - 41541s 275ms/step - loss: 2.3652
Epoch 4/7
151008/151008 [==============================] - 41518s 275ms/step - loss: 2.1662
Epoch 5/7
151008/151008 [==============================] - 41404s 274ms/step - loss: 2.0053
Epoch 6/7
151008/151008 [==============================] - 41322s 274ms/step - loss: 1.8754
Epoch 7/7
151008/151008 [==============================] - 41472s 275ms/step - loss: 1.7628


In [361]:
#model.save_weights('text_generator_400_0.2_400_0.2_400_0.2_1003.h5')
model.load_weights('text_generator_gigantic.h5')

In [326]:
#model.load_weights('text_generator_400_0.2_400_0.2_400_0.2_1003.h5')

In [378]:
string_mapped = X[3333]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]


In [379]:
len(X)

151008

In [380]:
txt=""
for char in full_string:
    txt = txt+char
txt

'h", \'inviting the next financial crisis . \', \'impeach trump: why republicans, not democrats are talking the protest . \', \'trump administration seportedly of the protest of the protest of the protest . \', \'trump administration seportedly of the protest of the protest of the protest . \', \'trump administration seportedly of the protest of the protest of the protest . \', \'trump administration seportedly of the protest of the protest of the protest . \', \'trump administration seportedly of the protest'

In [3]:

import keras
print(keras.__version__)

2.1.2


Using TensorFlow backend.


In [1]:
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


TensorFlow version: 1.4.1
